# End Station Finder

## The challenge:
My friend Lou has a project. They want to visit all the end stations on the Berlin S-Bahn and U-Bahn system. However, to count as a truly satisfying end station, the station must not only be at the end of its own line, but must also lack an interchange to another line. Lou wants only the end-iest of end stations.

I want to find the end stations and present them to Lou in a format they can use for their project.


## The data:

Berlin provides public transport data in the now-standard [GTFS](https://gtfs.org/) format, which stands for General (formerly Google) Transit Feed Specification. It is an [open standard](https://beyondtransparency.org/chapters/part-2/pioneering-open-data-standards-the-gtfs-story/) used by transit agencies to present their data in a uniform und useable way. It encompasses two types of data: realtime and static.
- The realtime data is what drives (pun intended) our on-the-go transit apps and notifies us of delays and changes.
- The static data (also known as *schedule data*) tells us and our apps about how all the points on the network are linked together. This is the GTFS data that will allow us identify the end stations.

### Static GTFS data format
GTFS is a data format in its own right, meant for "[GTFS-consuming applications](https://www.transitwiki.org/TransitWiki/index.php/Category:GTFS-consuming_applications)". However, conveniently, it is also easily readable in other software environements because it is merely a ZIP comprised of CSV (as TXT) files. The exact tables provided vary by agency. The following are the minimum required [table files for static GTFS](https://gtfs.org/schedule/reference/):
-  `stops.txt` showing stops' name and location
-  `stop_times.txt` showing every stop on every individual trip
-  `trips.txt` showing trips' route, direction, destination* (headsign), and more
-  `routes.txt` showing routes' transportation type, number, line colour, and more
-  `agency.txt`

My [GTFS download for Berlin](https://daten.berlin.de/datensaetze/vbb-fahrplandaten-gtfs) also contains these conditionally required and optional tables:
-  `frequencies.txt` (note: empty), `calendar.txt` and `calendar_dates.txt` showing service frequency, day and date availability
-  `pathways.txt` and `levels.txt` showing platform positions inside stations
-  `shapes.txt` showing a trip's path, expressed as a sequence of co-ordinates 
-  `transfers.txt` showing the connections between stations and details about these transfers

GTFS does not contain a simple list of `stops` on `route`. Instead, `stop_ID`s appear as sequences of `stop_times` in individual `trips`, which in turn belong to various `routes`. We need to join the IDs of stops, trips and routes to the (giant) `stop_times.txt`. 

I would therefore need to combine the following tables and fields to find the end stations if I were to make a database:

`stop_times.txt` for the sequence of stops
- "trip_id"         (primary key 1) (foreign key of trips.trip_id)
- "stop_sequence"   (primary key 2)
- "arrival_time"
- "departure_time"
- "stop_id" (foreign key of stops.stop_id)
- "pickup_type"
- "drop_off_type"

`transfers.txt` for the transfers between different routes (or in our case, the lack thereof)

- "from_stop_id"    (primary key 1) (foreign key of stops.stop_id)
- "to_stop_id"      (primary key 2) (foreign key 2 of stops.stop_id)
- "from_route_id"   (primary key 3) - might not be needed
- "to_route_id"     (primary key 4) - might not be needed
- "transfer_type"`  see [docs](https://gtfs.org/schedule/reference/#transferstxt)
- "min_transfer_time" 

`trips.txt` for matching the trip IDs to routes
- "trip_id"         (primary key)
- "route_id"        (foreign key of routes.route_id)
- "trip_headsign"   *
- "trip_short_name" (can be route number but appears not to be used for train numbers here)
- "direction_id"

`stops.txt` for matching the stop IDs to stops
- "stop_id"         (primary key)
- "stop_name"       
- "stop_lon"
- "stop_lat"

\* the "headsign" destination is not to be confused with the end of a line; some trips do not travel the whole route. In our data this is a text field rather than an ID.

## Tools for GTFS

There are some [packages designed to handle GTFS data](https://gtfs.org/resources/gtfs) in Python, created by various developers. These are mainly used for geographical and frequency data. 

However, I found three that I may be able to use to find the end stations in **Python**, more easily than by manually creating a database:

|    | Package | Data Structure | Potential Functions |
| ---| ------- | -------------- | ------------------- |
| 1.    | [`gtfs_kit`](https://mrcagney.github.io/gtfs_kit_docs/)   | Pandas  | `gtfs_kit.stops.get_stops` and `gtfs_kit.validators.check_transfers`|
| 2. | [`transit_service_analyst`](https://github.com/psrc/transit_service_analyst/wiki/transit_service_analyst-documentation) | Pandas | `get_line_stops_gdf()` |
| 3. | [`gtfs_utils`](https://open-bus-gtfs-utils.readthedocs.io/en/latest/route_stats.html)    | Pandas |  `route_stats` and `trip_stats`| 

I also noted down a few tools for **SQL**, in case the above don't work for me: 
1. [`node-gtfs`](https://github.com/BlinkTagInc/node-gtfs) SQLite
2. [`gtfs-via-postgres`](https://github.com/public-transport/gtfs-via-postgres) PostgreSQL
3. [`gtfs-schema`](https://github.com/tyleragreen/gtfs-schema) Schema builder


## `transit_service_analyst`
### Step 1: Load packages, extensions and data

In [1]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import plotly.express as px
import shapely.geometry
import numpy as np

I decided to install and try the `transit_service_analyst` package first. It also requires `geopandas` and `pandera` to be installed. Now I can import the module:

In [2]:
import transit_service_analyst as tsa

Then I used the command `tsa.load_gtfs(folder, service_date)` to try to load the data into a Pandas dataframe. The required arguments are:
- the location of the unzipped GTFS folder
- a sample date from the dataset in YYYYMMDD form. 

A date is needed because `tsa` constructs the routes based on the stops served in all scheduled trips on the sample day. A look in `calendar.txt` shows that my data runs from 20230615 to 20231209, so I picked today, 20230705, a Wednesday.

**Note on date and route sampling issues:**
Choosing only a single date to build the routes is a potential issue because there is the possibility that it might not sample the whole length of routes. For example, in the case of construction work that prevents trains reaching their normal end station, the scheduled trip data might show an point earlier than the end of the physical route. This can be partially controlled for using information already contained in Berlin's GTFS data. Indeed, today, like most or maybe all days, is marked as an exception in `calendar_dates.txt`. Later I will check the affected `service_id`s to find out if they are relevant. However, this would not account for any shortened routes that span enough time to be counted as the normal schedule rather than the exception, and I will attempt to overcome this later, probably by sampling various dates from the set.

In [ ]:
transit_analyst = tsa.load_gtfs('Berlin GTFS data', '20230705')

In [18]:
#i got an error: `Cannot set a DataFrame with multiple columns to the single column start_time_secs` [and other _time_secs columns]
#attempting to debug and reload. 
#Autoreload lets me use the changes instantly without a kernel reload.
%load_ext autoreload
%autoreload 2

### Failures due to data compatibility issues

#### Empty csv
Unfortunately it seems like this package requires a populated `frequencies.txt` to gather the `trip_id`s. This csv is however empty in Berlin's static data. The code in `gtfs_service.py` was set up to deal with the file existing or not existing, but not being empty:

```python
if os.path.exists(os.path.join(self.gtfs_dir, "frequencies.txt")):
            self.trips, self.stop_times = self.frequencies_to_trips()
```
Solved for now by renaming the file, but it would be good to suggest a change on Github later.

#### Inconsistent `route_type`s
Berlin's data uses [Google Transit extended route types](https://developers.google.com/transit/gtfs/reference/extended-route-types) rather than the classic [GTFS route types](https://gtfs.org/schedule/reference/#routestxt) so Pandera's schema validation fails:

```console
    Error Counts
    ------------
    - SchemaErrorReason.SCHEMA_COMPONENT_CHECK: 1
    
    Schema Error Summary
    --------------------
                                                                                        failure_cases  n_failure_cases
    schema_context column     check                                                                                   
    Column         route_type isin([0, 1, 2, 3, 4, 5, 6, 7, 11, 12])  [700, 100, 900, 1000, 109, 400]                6
```

Can I easily update the schema?

In [4]:
transit_analyst = tsa.load_gtfs('Berlin GTFS data', '20230705')
#routes_df failed Pandera schema validation and yielded another error so I added lazy=True to all dfs' validation in the source 

/home/sian/anaconda3/envs/stats_env/lib/python3.9/site-packages/transit_service_analyst/gtfs_service.py:95: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  trips_df = pd.read_csv(os.path.join(self.gtfs_dir, "trips.txt"))
/home/sian/anaconda3/envs/stats_env/lib/python3.9/site-packages/transit_service_analyst/gtfs_service.py:116: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  stop_times_df = pd.read_csv(os.path.join(self.gtfs_dir, "stop_times.txt"))


SchemaErrors: Schema Routes: A total of 1 schema errors were found.

Error Counts
------------
- SchemaErrorReason.SCHEMA_COMPONENT_CHECK: 1

Schema Error Summary
--------------------
                                                                                    failure_cases  n_failure_cases
schema_context column     check                                                                                   
Column         route_type isin([0, 1, 2, 3, 4, 5, 6, 7, 11, 12])  [700, 100, 900, 1000, 109, 400]                6

Usage Tip
---------

Directly inspect all errors by catching the exception:

```
try:
    schema.validate(dataframe, lazy=True)
except SchemaErrors as err:
    err.failure_cases  # dataframe of schema errors
    err.data  # invalid dataframe
```
